# AutoKoopman Overview

In [ ]:
import autokoopman as akm
import sympy as sp
import numpy as np
from autokoopman.benchmark import FitzHughNagumo

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

## Create Symbolic Systems


AutoKoopman has a `AutoKoopman.System` class, representing discrete and continuous time dynamical systems. Users can implement systems from python functions or symbolic expressions. Here, we will implement our own Lorenz Attractor. We start by introducing out state variables $\mathbf x = [x, y, z]$ and the evolution function $\dot{\mathbf x}$. Note how implementing the system is to writing LaTeX. We can pretty print the equation when we are done.

In [ ]:
# define our state variables using SymPy
x, y, z = sp.symbols("x y z")

# define xdot, our ODE expression
# note: see Lorenz attractor for more
sigma, rho, beta, tau = 10, 28, 8/3, 10.0
xdot = [
    (sigma * (y - x)) / tau,
    (x * (rho - z) - y) / tau,
    (x * y - beta *z) / tau
]

# pass these to a AutoKoopman System builder
my_sys = akm.symbolic_ode_system([x, y, z], xdot)

# display the equations in the notebook
my_sys.display_math()

## Benchmark Systems

AutoKoopman comes with a collection of benchmark systems, useful for generating training data and evaluating model performance. Many of these models are symbolically defined and we can display their functions here.

In [ ]:
# pull out the benchmark systems and display some information
import autokoopman.benchmark as abench
from autokoopman.core.system import SymbolicContinuousSystem
from IPython.display import Markdown, display

for k, v in abench.__dict__.items():
    try:
        if issubclass(v, SymbolicContinuousSystem):
            inst = v()
            display(Markdown(f"### {k}"))
            display(Markdown(f"Dimensions: {inst.dimension}"))
            display(Markdown(f"State Names: {inst.names}"))
            inst.display_math()
    except:
        pass

## System Simulation

Once we have an `AutoKoopman.System` implemented, we can simulate trajectories by solving the initial value problem (IVP). Given these argument, we sample the IVP solution uniformly over a timespan with a given sampling period.

In [ ]:
# let's build a dataset from this benchmark system
initial_conditions = np.random.uniform(low=-20.0, high=20.0, size=(30, 3))

trajs = my_sys.solve_ivps(
    initial_conditions.tolist(), 
    tspan=[0.0, 10.0], 
    sampling_period=0.05
)

trajs

In [ ]:
plt.figure(figsize=(12, 8))

# system is 3d, so let's only look at the first two
visible_dims = (0, 1)

# iterate over trajectories
for traj in trajs:
    plt.plot(*traj.states[:, visible_dims].T, 'black')
    
# label the plot
plt.title("My Lorenz Attractor System Trajectories")
plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.show()

## Learn Systems from Data

In [ ]:
# the autokoopman convenience function
# one function to optimize hyperparameters and learn a system
from autokoopman import auto_koopman


# learn a system
#(0.0001, 161)
results = auto_koopman(
    trajs,
    sampling_period=0.05,
    obs_type="rff",
    opt="grid",
    n_obs=200,
    max_opt_iter=200,
    grid_param_slices=2,
    n_splits=5,
    #rank=(1, 200, 20)
    
    # ELEW: tmp for fast run
    rank=(161, 162, 1),
    lengthscale=(0.0001, 0.00011)
)

model = results['tuned_model']

## Model Evaluation

In [ ]:
# construct test initial conditions different from the training one
initial_conditions_test = np.random.uniform(low=-10.0, high=10.0, size=(10, 3))

# create ground truth trajectories and predictions using our tuned model
test_trajs = my_sys.solve_ivps(initial_conditions_test.tolist(), tspan=[0.0, 10.0], sampling_period=0.05)
pred_trajs = model.solve_ivps(initial_conditions_test.tolist(), tspan=[0.0, 10.0], sampling_period=0.05)


In [ ]:
# error plot
plt.figure(figsize=(15, 7))

# for trajectories that have the same names and time points, we can do arithemtic on them
# substract the two trajectories and get their norm for error
diff_trajs = (test_trajs - pred_trajs).norm()

# plot the norm of the difference
for t in diff_trajs:
    plt.plot(t.states, 'k', alpha=0.8)
    
plt.ylabel("Absolute Prediction Error")
plt.xlabel("Time Step [n]")
plt.title("Model Prediction Error vs Time")
plt.grid()

In [ ]:
# plot the results
import matplotlib.pyplot as plt

# build a multi-plot
fig, ax = plt.subplots(figsize=(15, 20), nrows=2, sharex=True)


# iterate through the trajectories to plot them
for idx, t in enumerate(test_trajs):
    ax[0].plot(*t.states[:, (0, 1)].T, 'k', label="Ground Truth" if idx == 0 else None)
    ax[1].plot(*t.states[:, (0, 2)].T, 'k', label="Ground Truth" if idx == 0 else None)

for idx, t in enumerate(pred_trajs):
    ax[0].plot(*t.states[:, (0, 1)].T, 'r', label="Model Prediction" if idx == 0 else None)
    ax[1].plot(*t.states[:, (0, 2)].T, 'r', label="Model Prediction" if idx == 0 else None)

    
# label things
ax[0].grid()
ax[1].grid()
ax[0].legend()
plt.tight_layout()
ax[0].set_title("AutoKoopman Learned System")
ax[1].set_xlabel("x")
ax[0].set_ylabel("y")
ax[1].set_ylabel("z")
plt.show()

## System Linearization

One benefit of learning systems using Koopman operator techniques is that they are linear koopman observables (Koopman linearized).

In [ ]:
# plot the eigenvalues of the learned system
plt.figure(figsize=(8, 8))

for v in model.eigenvalues:
    x, y = np.real(v), np.imag(v)
    plt.scatter(x, y, marker="x", color="k")

# plot the unit circle for reference
circle = plt.Circle((0.0, 0.0), 1.0, fill=False, color="r")
plt.gca().add_patch(circle)
plt.xlabel("Re")
plt.ylabel("Im")
plt.title("Eigenvalues of the Koopman Linearized Model")
plt.grid()